# PRACTICA GUIADA: Introducción a Machine Learning 1

Ahora revisaremos varios ejemplos simples de aplicación de métodos de aprendizaje supervisado y no supervisado

### Ejemplo de aprendizaje supervisado: Regresión lineal simple

Como ejemplo de este proceso, vamos a considerar una regresión lineal simple, es decir, el caso común de ajustar una línea a datos de la forma $(x, y)$.

Vamos a generar el siguiente dataset para nuestro ejemplo de regresión:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline 

rng = np.random.RandomState(42)
x = 10 * rng.rand(50)
y = 2 * x - 1 + rng.randn(50)

# Ploteamos
plt.scatter(x, y);

Una vez generados los datos, podemos usar la receta estudiada anteriormente. Vamos a recorrer los pasos del proceso: 

#### 1. Seleccionar una "clase de modelo"

En Scikit-Learn, cada clase de modelo se representa con una clase de Python. 

Entonces, por ejemplo, si queremos computar un modelo de regresión lineal simple, podemos importar la clase de regresión lineal de esta forma:

In [ ]:
from sklearn.linear_model import LinearRegression

Notar que también existen otros modelos de regresión lineal más generales; Podés leer más acerca de ellos en la [documentación ``sklearn.linear_model``](http://Scikit-Learn.org/stable/modules/linear_model.html). 

#### 2. Elegir los hiperparámetros del modelo

Es importante destacar que *una clase de modelo no es lo mismo que una instancia de modelo*.

Una vez que hemos decidido nuestra clase de modelo, todavía tenemos que tomar algunas decisiones. Dependiendo de la clase de modelo con la que trabajemos, podríamos tener que responder a una o más preguntas como las siguientes:

- ¿Queremos ajustar también un intercepto (intercept = True)?
- ¿Queremos que el modelo esté normalizado?
- ¿Queremos agregar features calculados a partir del input para darle mayor flexibilidad al modelo?
- ¿Qué grado de "regularización" vamos a querer usar en el modelo?

Estos son ejemplos de las importantes decisiones que deben hacerse **una vez que hemos seleccionado la clase de modelo a usar**.

Estas elecciones se representan frecuentemente como *hiperparámetros*, o parámetros que deben ser seteados antes de que el modelo sea ajustado a los datos. 

En Scikit-Learn, los hiperparámetros son elegidos como argumentos en la instanciación del modelo. Exploraremos cómo podemos justificar cuantitativamente la elección de hiperparámetros en las próximas clases.  

Para nuestro ejemplo de regresión lineal, podemos instanciar la clase ``LinearRegression`` y especificar que nos gustaría ajustar el intercepto usando el hiperparámetro ``fit_intercept``: 

In [ ]:
model = LinearRegression(fit_intercept=True)
model

**Tener en cuenta**: cuando el modelo es instanciado, la única acción que sucede es almacenar los valores de estos valores de hiperparámetros.

En particular, todavía no hemos aplicado el modelo a ningún dato: la API de Scikit-Learn hace una distinción muy clara entre la *elección del modelo con sus hiperparámetros* y la *aplicación del modelo a los datos*. 

#### 3. Preparar los datos en una matriz de features y un vector de target

Previamente hemos hablado de la representación de datos de Scikit-Learn, la cual requiere una matriz de features de dos dimensiones y un vector target de una dimensión.

Aquí nuestra variable target ``y`` ya se encuentra en un formato correcto (un array de longitud ``n_samples``), pero necesitamos procesar los datos en ``x`` para convertirlos en una matriz de tamaño ``[n_samples, n_features]``.

En este caso, esto se reduce a un simple cambio de forma (reshaping) del array de una dimensión: 

In [ ]:
X = x[:, np.newaxis]
X.shape

# Otra forma de hacerlo
# X = x.reshape(-1,1)

#### 4. Ajustar el modelo a los datos

Ahora es momento de aplicar nuestro modelo a los datos.
Esto puede hacerse con el método ``fit()`` de nuestra instancia de modelo.

In [ ]:
model.fit(X, y)

El método ``fit()`` realiza una secuencia de cómputos internos dependientes del modelo, y los resultados de estas operaciones son almacenadas en atributos específicos de la clase de modelo que el usuario luego puede explorar.

En Scikit-learn, por convención, todos los atributos que representan los parámetros de los modelos que fueron aprendidos durante el procesos de entrenamiento con fit ``fit()``, tienen `underscores` en sus nombres; por ejemplo en este modelo lineal, podemos observar el parámetro coef_ y el parámetro intercept_:

In [ ]:
model.coef_

In [ ]:
model.intercept_

Estos dos parámetros representan la pendiente y el intercepto del ajuste lineal simple a los datos. 
Comparándolos con la definición del dataset sintético creado al principio del ejemplo, podemos ver que están muy cerca de la pendiente 2 y el intercepto -1 definidos en la fórmula para generar los datos.

Una pregunta que surge frecuentemente se relaciona con incertidumbre o incerteza (uncertainty) en estos parámetros internos del modelo. 

En general, Scikit-Learn no provee herramientas para obtener conclusiones del estado interno de los modelos: interpretar los parámetros de un modelo tiene mucho más que ver con una pregunta de *modelado estadístico* más que una pregunta de *machine learning*.

Machine learning en cambio se enfoca en la calidad con la cual el modelo *predice*.

Si te interesa investigar el significado de los parámetros de ajuste dentro del modelo, existen otras herramientas, incluyendo el paquete de python [Statsmodels](http://statsmodels.sourceforge.net/).

#### 5. Predecir etiquetas para datos desconocidos

Una vez que el modelo es entrenado, la principal tarea en el aprendizaje supervisado es evaluarlo en base a lo que dice acerca de nuevos datos que no fueron parte del **set de entrenamiento**. 

En Scikit-Learn, esto puede hacerse usando el método ``predict()``. 
En el contexto de este ejemplo, nuestro "nuevo dataset" será un vector de valores *x*, y observaremos los valores *y* que el modelo predice.

In [ ]:
xfit = np.linspace(-1, 11)

Como antes, necesitamos manipular estos valores *x* en una matriz de features de tamaño ``[n_samples, m_features]``, y recién despueś de este paso podremos utilizarla como argumento para la predicción. 

In [ ]:
Xfit = xfit[:, np.newaxis]
yfit = model.predict(Xfit)

Finalmente, vamos a visualizar los resultados ploteando primero los datos originales, y luego el ajuste de nuestro modelo lineal:

In [ ]:
# me parece que se olvida completar el nuevo y
# y = 2 * x - 1 + rng.randn(50)

plt.scatter(x, y)
plt.plot(xfit, yfit);

Típicamente, la eficacia del modelo es evaluada comparando sus resultados con algún baseline conocido, como veremos en el próximo ejemplo.

### Ejemplo de aprendizaje supervisado: Clasificación con el dataset Iris

Veamos otro ejemplo de este proceso, usando el dataset Iris del que hablamos previamente. 
Nuestra pregunta será esta: dado un modelo entrenado en una porción del del dataset Iris, cuán bien podemos predecir las etiquetas restantes?.

In [ ]:
import seaborn as sns
iris = sns.load_dataset('iris')
iris.head()

In [ ]:
X_iris = iris.drop('species', axis=1)
X_iris.shape

In [ ]:
y_iris = iris['species']
y_iris.shape

Para esta tarea, usaremos un modelo generativo extremadamente simple conocido como Naive Bayes Gausiano, el cuál procede asumiendo que cada clase se construye a partir de una distribución Gausiana. Lo veremos en detalle más adelante en el curso.

Porque es muy rápido y no tiene hiperparámetros para elegir, Naive Bayes Gausiano es frecuentemente un buen modelo para usar como una clasificación baseline, antes de explorar si pueden encontrarse mejoras a través de modelos más sofisticados.

Nos gustaría evaluar el modelo en datos que no hayan sido usados en el entrenamiento, por lo tanto vamos a dividir los datos en un *training set* y un *testing set*.

Esto podría hacerse a mano, pero es más conveniente usar la función ``train_test_split``.

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X_iris, y_iris,
                                                random_state=1)

Con los datos preparados, podemos seguir nuestra receta para predecir las etiquetas:

In [ ]:
from sklearn.naive_bayes import GaussianNB # 1. elegir la clase de modelo
model = GaussianNB()                       # 2. instanciar el modelo
model.fit(Xtrain, ytrain)                  # 3. ajustar el modelo a los datos
y_model = model.predict(Xtest)             # 4. predecir a partir de nuevos datos

Finalmente, podemos usar la función ``accuracy_score`` para estudiar la proporción de etiquetas predichas que coinciden con el valor de verdad correspondiente a esa observación.

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest, y_model)

Con un accuracy que supera el 97%, podemos ver que incluso este sencillo algoritmo de clasificación es efectivo para este dataset particular.

### Ejemplo de aprendizaje no supervisado: Dimensionalidad de Iris

Como un ejemplo de un problema de aprendizaje no supervisado, veamos cómo reducir la dimensionalidad de los datos de Iris para poder visualizarlos más fácilmente. 

Recordemos que el dataset Iris es cuatridimensional: hay cuatro features medidas para cada observación (sample).

La tarea de reducción de la dimensionalidad es investigar si hay una representación apropiada de baja dimensionalidad que retiene las características esenciales del dataset original. 

Frecuentemente la reducción de la dimensionalidad se usa como una ayuda para visualizar datos: después de todo es mucho más fácil plotear datos en dos dimensiones que en cuatro o más dimensiones. 

En este ejemplo vamos a usar Principal Component Analysis (PCA), que es una técnica rápida de reducción lineal de la dimensionalidad. 
Vamos a pedirle al modelo que devuelva dos componentes, es decir, una representación bidimensional de los datos. 

Siguiendo la secuencia de pasos presentada previamente, tenemos:

In [ ]:
# 1. Seleccionar la clase de modelo
from sklearn.decomposition import PCA  

# 2. Instanciar el modelo con hiperparámetros
model = PCA(n_components=2)            

# 3. Ajustar a los datos. Notar que no especificamos "y" 
model.fit(X_iris)                      

# 4. Transformar los datos a dos dimensiones
X_2D = model.transform(X_iris)         

Ahora vamos a plotear los resultados. Una forma rápida de hacer esto es insertar los resultados en el ``DataFrame`` original de Iris, y usar el método ``lmplot`` de Seabor para mostrar los resultados:


In [ ]:
iris['PCA1'] = X_2D[:, 0]
iris['PCA2'] = X_2D[:, 1]
sns.lmplot("PCA1", "PCA2", hue='species', data=iris, fit_reg=False);

Vemos que en la representación en dos dimensiones, las especies están relativamente bien separadas, incluso aunque el algoritmo PCA no tenía conocimiento de las etiquetas de las especies de flores! 

Esto nos indica que una clasificación relativamente sencilla debería probablemente ser efectiva con este dataset, como vimos antes. 

### Aprendizaje no supervisado: Clustering con Iris

Vamos a aplicar un algoritmo de clustering al dataset Iris.

Un algoritmo de clustering intenta encontrar grupos distintos sin tener referencias a etiquetas en los datos. 

Vamos a usar un método poderoso de clustering llamado Gaussian mixture model (GMM). 
Un GMM intenta modelar los datos como una colección de blobs Gausianos. 

Podemos ajustar el GMM de la siguiente forma:

In [ ]:
# 1. Elegimos la clase de modelo
from sklearn.mixture import GaussianMixture

# 2. Instanciamos el modelo con sus hiperparámetros
model = GaussianMixture(n_components=3,
            covariance_type='full')  

# 3. Ajustamos a los datos. Notar que "y" no es especificada
model.fit(X_iris)                    

# 4. Determinamos las etiquetas de los clusters
y_gmm = model.predict(X_iris)       

Como antes, agregaremos las etiquetas de los clusters al ``DataFrame`` Iris y usaremos Seaborn para plotear los resultados:

In [ ]:
iris['cluster'] = y_gmm
sns.lmplot("PCA1", "PCA2", data=iris, hue='species',
           col='cluster', fit_reg=False);

Al separar los datos por número de cluster, vemos exactamente cuán bien el algoritmo GMM ha recuperado la etiqueta subyacente: la especie *setosa* es separada perfectamente dentro del cluster 0, aunque vemos una pequeña porción mezclada entre *versicolor* y *virginica*. 

Esto significa que incluso sin un experto que nos diga las etiquetas de las flores individuales, las medidas de estas observaciones son lo suficientemente distintas para que podamos identificar *automáticamente* la prsencia de estos diferentes grupos de especies con un simple algoritmo de clustering. 

Este tipo de algoritmo podría incluso darle a los expertos en la disciplina algunas pistas sobre las relaciones entre las muestras que están observando (??).

## En resumen

En esta sección hemos cubierto las características esenciales de la representación de datos en Scikit-Learn y la API de estimadores. 

Sin importar el tipo de estimador, el mismo patrón de importar/instanciar/fittear/predecir se mantiene en todos los casos. 

Armado con esta nueva información sobre la API de estimadores, vos podés explorar la documentación de Scikit-Learn y comenzar a probar varios modelos sobre tus datos. 

En la próxima clase, vamos a explorar el que tal vez es el tópico más importante en machine learning: cómo seleccionar y validar tu modelo. 